<a href="https://colab.research.google.com/github/frzlh/DeepLearning/blob/main/transformLearing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!wget https://github.com/lucidfrontier45/PyTorch-Book/raw/master/data/taco_and_burrito.tar.gz

--2024-08-20 08:44:29--  https://github.com/lucidfrontier45/PyTorch-Book/raw/master/data/taco_and_burrito.tar.gz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lucidfrontier45/PyTorch-Book/master/data/taco_and_burrito.tar.gz [following]
--2024-08-20 08:44:29--  https://raw.githubusercontent.com/lucidfrontier45/PyTorch-Book/master/data/taco_and_burrito.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15292798 (15M) [application/octet-stream]
Saving to: ‘taco_and_burrito.tar.gz.8’

taco_and_burrito.ta 100%[===================>]  14.58M  --.-KB/s    in 0.04s   

2024-08-20 08:44:30 (350 

In [57]:
!tar -zxvf taco_and_burrito.tar.gz

taco_and_burrito/
taco_and_burrito/test/
taco_and_burrito/test/burrito/
taco_and_burrito/test/burrito/363.jpg
taco_and_burrito/test/burrito/372.jpg
taco_and_burrito/test/burrito/362.jpg
taco_and_burrito/test/burrito/386.jpg
taco_and_burrito/test/burrito/379.jpg
taco_and_burrito/test/burrito/367.jpg
taco_and_burrito/test/burrito/361.jpg
taco_and_burrito/test/burrito/384.jpg
taco_and_burrito/test/burrito/377.jpg
taco_and_burrito/test/burrito/365.jpg
taco_and_burrito/test/burrito/368.jpg
taco_and_burrito/test/burrito/374.jpg
taco_and_burrito/test/burrito/388.jpg
taco_and_burrito/test/burrito/378.jpg
taco_and_burrito/test/burrito/380.jpg
taco_and_burrito/test/burrito/370.jpg
taco_and_burrito/test/burrito/364.jpg
taco_and_burrito/test/burrito/385.jpg
taco_and_burrito/test/burrito/381.jpg
taco_and_burrito/test/burrito/383.jpg
taco_and_burrito/test/burrito/373.jpg
taco_and_burrito/test/burrito/366.jpg
taco_and_burrito/test/burrito/389.jpg
taco_and_burrito/test/burrito/376.jpg
taco_and_burrito

In [58]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch
import tqdm
from torch import nn,optim
from torch.utils.data import (Dataset,DataLoader,TensorDataset)

train_imgs=ImageFolder(
    "taco_and_burrito/train/",
    transform=transforms.Compose(
        [transforms.RandomCrop(224),
         transforms.ToTensor()]
    )
)

test_imgs=ImageFolder(
    "taco_and_burrito/test/",
    transform=transforms.Compose(
        [transforms.CenterCrop(224),
         transforms.ToTensor()]
    )
)

train_loader=DataLoader(train_imgs,batch_size=32,shuffle=True)
test_loader=DataLoader(test_imgs,batch_size=32,shuffle=False)

# print(train_imgs.classes)
# print(train_imgs.class_to_idx)


In [59]:
from torchvision import models
net=models.resnet18(pretrained=True)

for p in net.parameters():
  p.requires_grad=False

fc_input_dim=net.fc.in_features
net.fc=nn.Linear(fc_input_dim,2)

按原样输入输出创建一个虚拟层，并进行fc层的替换

In [60]:
# from torchvision import models
# class IdentityLayer(nn.Module):
#   def forward(self,x):
#     return x
# net=models.resnet18(pretrained=True)
# for p in net.parameters():
#   p.requires_grad=False
# net.fc=IdentityLayer()

In [61]:
def eval_net(net,data_loader,device="cpu"):
  net.eval()
  ys=[]
  ypreds=[]
  for x,y in data_loader:
    x=x.to(device)
    y=y.to(device)
    with torch.no_grad():
      _,y_pre=net(x).max(1)
    ys.append(y)
    ypreds.append(y_pre)
  ys=torch.cat(ys)
  ypreds=torch.cat(ypreds)
  acc=(ys==ypreds).float().sum()/len(ys)
  return acc.item()

In [62]:
def train_net(net,train_loader,test_loader,only_fc=True,optimizer_cls=optim.Adam,loss_fn=nn.CrossEntropyLoss(),n_iter=10,device="cpu"):
  train_losses=[]
  train_acc=[]
  val_acc=[]
  if only_fc:
    optimizer=optimizer_cls(net.fc.parameters())
  else:
    optimizer=optimizer_cls(net.parameters())

  for epoch in range(n_iter):
    running_loss=0.0
    net.train()
    n=0
    n_acc=0
    for i,(xx,yy) in tqdm.tqdm(enumerate(train_loader),total=len(train_loader)):
      xx=xx.to(device)
      yy=yy.to(device)
      h=net(xx)
      loss=loss_fn(h,yy)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()
      n+=len(xx)
      _,y_pred=h.max(1)
      n_acc+=(yy==y_pred).float().sum().item()
    train_losses.append(running_loss/i)
    train_acc.append(n_acc/n)
    val_acc.append(eval_net(net,test_loader,device))
    print(epoch,train_losses[-1],train_acc[-1],val_acc[-1],flush=True)

In [63]:
net.to("cuda:0")
train_net(net,train_loader,test_loader,n_iter=20,device="cuda:0")

100%|██████████| 23/23 [00:02<00:00, 11.16it/s]


0 0.6945208473639055 0.6137640449438202 0.7000000476837158


100%|██████████| 23/23 [00:02<00:00,  8.09it/s]


1 0.5903545753522352 0.7008426966292135 0.8166667222976685


100%|██████████| 23/23 [00:03<00:00,  6.98it/s]


...

17 0.3244797323237766 0.8679775280898876 0.8833333849906921


100%|██████████| 23/23 [00:01<00:00, 12.85it/s]


18 0.30186817862770776 0.8862359550561798 0.8166667222976685


100%|██████████| 23/23 [00:01<00:00, 13.27it/s]


19 0.37078517641533504 0.8441011235955056 0.7500000596046448
